In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (12, 6)

In [ ]:
DATA_PATH = "/content/processed_flight_records.csv"

df = pd.read_csv(DATA_PATH)

# Identify key columns
date_cols = [c for c in df.columns if 'date' in c.lower() or 'time' in c.lower()]
status_cols = [c for c in df.columns if 'status' in c.lower()]
delay_cols = [c for c in df.columns if 'delay' in c.lower()]

# Assign columns
date_col = date_cols[0]
status_col = status_cols[0] if status_cols else None
delay_col = delay_cols[0] if delay_cols else None

# Parse datetime
df[date_col] = pd.to_datetime(df[date_col])

In [ ]:
if status_col:
    df['is_delayed'] = df[status_col].astype(str).str.lower().str.contains('delay')
    df['is_cancelled'] = df[status_col].astype(str).str.lower().str.contains('cancel')
else:
    df['is_delayed'] = False
    df['is_cancelled'] = False

# Time aggregation keys
df['month'] = df[date_col].dt.to_period('M')
df['week'] = df[date_col].dt.to_period('W')

In [ ]:
monthly_kpis = df.groupby('month').agg(
    total_flights=('is_delayed', 'count'),
    delay_rate=('is_delayed', 'mean'),
    cancellation_rate=('is_cancelled', 'mean'),
    avg_delay_minutes=(delay_col, 'mean') if delay_col else ('is_delayed', 'mean')
)

monthly_kpis['delay_rate'] *= 100
monthly_kpis['cancellation_rate'] *= 100

monthly_kpis.head()


In [ ]:
plt.figure()
monthly_kpis['delay_rate'].plot(label='Delay Rate (%)')
monthly_kpis['cancellation_rate'].plot(label='Cancellation Rate (%)')
plt.legend()
plt.title("Monthly Delay and Cancellation Rates")
plt.ylabel("Rate (%)")
plt.xlabel("Month")
plt.show()

if delay_col:
    plt.figure()
    monthly_kpis['avg_delay_minutes'].plot()
    plt.title("Average Delay Minutes Over Time")
    plt.ylabel("Minutes")
    plt.xlabel("Month")
    plt.show()

In [ ]:
rolling_window = 3
rolling_kpis = monthly_kpis.rolling(window=rolling_window).mean()

plt.figure()
monthly_kpis['delay_rate'].plot(alpha=0.4, label='Actual')
rolling_kpis['delay_rate'].plot(label=f'{rolling_window}-Month Rolling Avg')
plt.legend()
plt.title("Delay Rate Stability Over Time")
plt.ylabel("Delay Rate (%)")
plt.show()

In [ ]:
airline_cols = [c for c in df.columns if 'airline' in c.lower() or 'icao' in c.lower()]

if airline_cols:
    airline_col = airline_cols[0]
    airline_monthly = df.groupby([airline_col, 'month']).agg(
        delay_rate=('is_delayed', 'mean'),
        cancellation_rate=('is_cancelled', 'mean')
    ) * 100

    airline_monthly.head()

In [ ]:
volatility = monthly_kpis[['delay_rate', 'cancellation_rate']].std()
volatility


In [ ]:
monthly_kpis.to_csv("/content/monthly_performance_kpis.csv")
rolling_kpis.to_csv("/content/rolling_performance_kpis.csv")

print("Performance metrics over time outputs saved.")